https://greendreamtrre.tistory.com/268

* 기본 url 존재: 상세 page에서 확인
* 해당 기본 url에 '필수 요청사항'들을 더하여 request url 완성
* 필수 요청사항 및 상세사항은 상세 문서 확인 필요

In [5]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [6]:
def get_data(api_key_p, rgn_cd_p, date_p) :
    queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
        {
            quote_plus('ServiceKey') : api_key_p,    # 필수 항목 1 : 서비스키 (본인의 서비스키)
            quote_plus('LAWD_CD') : rgn_cd_p,          # 필수 항목 2 : 지역코드 (법정코드목록조회에서 확인)
            quote_plus('DEAL_YMD') : date_p,        # 픽수 항목 3 : 계약월
            quote_plus('numOfRows') : '9999'
         }
    )

    response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
    
    return response

def get_items(reponse) : 
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    rows = xmlobj.findAll('item')

    rowList = []
    nameList = []
    columnList = {}
    rowsLen = len(rows)
    
    for i in range(rowsLen):
        columns = rows[i].find_all()

        columnsLen = len(columns)
        for j in range(0, columnsLen):

            #### 1. Column Name 만들기
            columnList[columns[j].name] =  columns[j].text

            #### 2. Column 별 data 수집하기 (Row 데이터 생성)
            #eachColumn = columns[j].text # column 내 순서대로 값들 추출
            #columnList.append(eachColumn) # 추출한 값들을 list 형태로 저장, 다 저장되면 1개 row의 모든 값들 저장 완성

        #### 3. Row 데이터 누적
        rowList.append(columnList) # 위에서 완성된 row data를 rowList에 저장하여 1개씩 row 누적 완성

        columnList = {}

    result = pd.DataFrame(rowList)
    int_col = ['거래금액', '건축년도', '년', '전용면적', '월', '일', '층']
    
    for i in int_col :
        result[i] = pd.to_numeric(result[i].str.replace(',',''))
    
    return result

## 1. API를 통해 데이터 다운로드하는 법

In [7]:
xmlUrl = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
api_key = unquote('')

In [8]:
############# 필수 항목은 공공데이터 상세설명 -> 상세기능 -> 요청변수 중 '필수' 항목

queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : api_key,    # 필수 항목 1 : 서비스키 (본인의 서비스키)
        quote_plus('LAWD_CD') : '11110',          # 필수 항목 2 : 지역코드 (법정코드목록조회에서 확인)
        quote_plus('DEAL_YMD') : '201512'         # 픽수 항목 3 : 계약월
     }
)

In [9]:
# ## Decode 후 과정
# api_key_decode = requests.utils.unquote(My_API_Key)
# parameters = {'ServiceKey' : api_key_decode, 'LAWD_CD' :'11110', 'DEAL_YMD':'201512'}
# req = requests.get(xmlUrl, params = parameters)
# req.text

In [10]:
response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
# xmlobj    # 디버깅용.

In [11]:
rows = xmlobj.findAll('item') # 전체 item 데이터 보유
columns = rows[0].find_all() # 1개 item의 데이터
columns[0].name, columns[0].text # 첫번째 item의 name, text 추출

IndexError: list index out of range

In [ ]:
rows[:3]

In [ ]:
columns

In [ ]:
columns[0].name, columns[0].text

In [ ]:
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        
        #### Column Name 만들기
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (데이터 형태는 동일하므로)
        if i == 0:
            nameList.append(columns[j].name)
            
        #### Column 별 data 수집하기 (Row 데이터 생성)
        eachColumn = columns[j].text # column 내 순서대로 값들 추출
        columnList.append(eachColumn) # 추출한 값들을 list 형태로 저장, 다 저장되면 1개 row의 모든 값들 저장 완성
    
    ### Row 데이터 누적
    rowList.append(columnList) # 위에서 완성된 row data를 rowList에 저장하여 1개씩 row 누적 완성
    
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList) # 값 자체가 row로 누적되었고, nameList는 column 명임
result.head()

## 2. 원하는 지역구 매매 확인

* 법정도 코드는 오른쪽에서 다운 받으면 된다. https://www.code.go.kr/index.do

In [ ]:
rgn_cd = pd.read_csv('region_code.txt', encoding='euc-kr', delimiter='\t')
rgn_nm = '강남구'
rgn_nb = rgn_cd.loc[rgn_cd['법정동명'].str.contains(rgn_nm),'법정동코드'].iloc[0, ]
rgn_cd_p = str(rgn_nb)[:5]

In [ ]:
response = get_data(api_key, rgn_cd_p, '202012')
res_df = get_items(response)
res_df.shape

In [ ]:
res_df.head()

In [ ]:
res_df['전용면적'].plot.hist(bins= 100)

In [ ]:
# https://ai-creator.tistory.com/24

def get_data(gu_code, base_date):
    url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?"
    service_key = "<인증키를 입력하세요>"    
    payload = "LAWD_CD=" + gu_code + "&" + \
              "DEAL_YMD=" + base_date + "&" + \
              "serviceKey=" + service_key + "&"

    res = requests.get(url + payload)
    
    return res

def get_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list